In [6]:
from nltk.tokenize.treebank import TreebankWordTokenizer
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import json
import sys

[nltk_data] Downloading package omw-1.4 to C:\Users\Lola Monroy
[nltk_data]     Mir\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Lola Monroy
[nltk_data]     Mir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Lola Monroy
[nltk_data]     Mir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Lola Monroy
[nltk_data]     Mir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
with open('house_data.json') as f:
  data = json.load(f)

In [8]:
data['houses'].append({'id': 26, 'type': 'sale', 'bedrooms': '5', 'bathrooms': '3', 'price': '200k', 'square_meters': '90', 'floor': '6', 'elevator': 'Yes', 'commercial_use': 'Yes', 'terrace': 'Yes', 'location': 'Tarragona'})

id = 26
house = data['houses'][id-1]

In [9]:
def print_question(prompt, possible_options = []):
    print(prompt)  # El chatbot imprimeix la pregunta 
    if not len(possible_options) == 0:  # En cas que la llista d'opcions NO estigui buida -> ha d'oferir opcions
      print("Options:", ", ".join(possible_options)) # Ofereix a l'usuari les possibles respostes 

def initialize_available_options(house_data, available_options): # available_options és un dict 
    for house in house_data['houses']: 
        for key, value in house.items():  # Per cada característica de cada casa
            available_options.setdefault(key, set()).add(value)  # Guarda un set per cada característica amb totes les opcions que hi ha per cada una (un set per no tenir repetits)

def preprocess_answer(answer): # Tokenitza la resposta del usuari
    answer = nltk.word_tokenize(answer)
    return answer 

def get_numerical_value(tok_answer): # Extreu valors numèrics de la resposta tokenitzada amb la funció anterior
    for token in tok_answer:
        if token.isnumeric() or token[:-1].isnumeric(): # Mira si el token és numèric o si sense l'ultim char ho és (per treure 'k' de 35k)
            return token 
    return '' # Només retorna un valor numèric si el troba, sinó retorna '' osigui res

In [10]:
def process_numerical_question(question): # question és un diccionari -> un item dins de data['questions']
    print_question(question['question']) 
    while True: # Bucle fins que l'usuari retorni una resposta vàlida
      answer = input(question['prompt']) # Espera la resposta de l'usuari, i espera que la resposta sigui el prompt corresponent
      tok_answer = preprocess_answer(answer) # Tokenitza la resposta rebuda
      value = get_numerical_value(tok_answer)
      if not value == '': 
        return value # Retorna el valor numèric obtingut si en troba un

def process_multichoice_question(question, options):
    print_question(question['question'], options) # Li dona al user la pregunta i les opcions
    while True:
      answer = input(question['prompt'])
      if answer in options: # Verifica si és opció vàlida
        return answer

In [12]:
print(data['start_message']) 

user_preferences, available_options = {}, {} 
initialize_available_options(data, available_options) 

for question in data['questions']:
  answer_key = question['answer_key'] 
  possible_options = list(available_options.get(answer_key))

  if question['type'] == 'numerical':
    answer = process_numerical_question(question)
  else:
    answer = process_multichoice_question(question, possible_options)

  user_preferences[answer_key] = answer 

Welcome to the House Buying Assistant!
How many bedrooms do you need?
How many bathrooms do you need?
What is your budget for the house?
How many square meters do you need?
Which city or neighborhood would you prefer?
Options: L'Hospitalet de Llobregat, Barcelona, Santa Coloma de Gramenet, Esplugues de Llobregat, Tarragona
